- #### Transformaciones
    - ##### agg
    - ##### describe
    - ##### summary
    - ##### crossJoin
    - ##### sample
    - ##### na functions
    - ##### UDF

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_10") \
        .master("local[*]") \
        .getOrCreate()

In [2]:
base_path = "../resources/data/tmp/parquet/"
movies_df = spark.read.parquet(base_path + "09/movies")
top_movies_df = spark.read.parquet(base_path + "09/top_movies")
movies_df.show(1)

+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|movie_id|           title|              genres|year|           tag_count|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|[{1990S, 1}, {200...|2006-01-12 19:19:35|      3.89|         0.93|       76813|1996-01-28 18:00:00|
+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
only showing top 1 row



In [5]:
movies_df.count()

79477

In [6]:
movies_df.printSchema()

root
 |-- movie_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- year: integer (nullable = true)
 |-- tag_count: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- tag: string (nullable = true)
 |    |    |-- count: long (nullable = true)
 |-- min_time_tag: timestamp (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- stddev_rating: double (nullable = true)
 |-- count_rating: long (nullable = true)
 |-- min_time_rating: timestamp (nullable = true)



### agg

In [7]:
import pyspark.sql.functions as f
movies_df.agg(
    f.count("movie_id").alias("count_movie_id"),
    f.approx_count_distinct(f.col("movie_id")).alias("approx_count_movie_id"),
    f.count_distinct(f.col("movie_id")).alias("count_d_movie_id"),
    f.min("min_time_tag").alias("min_time_tag"),
    f.min("min_time_rating").alias("min_time_rating"),
    f.max("count_rating").alias("max_count_rating"),
    f.max(f.size("tag_count")).alias("max_tags")
).show()

## equivalente a DataFrame.groupBy().agg( ... )

+--------------+---------------------+----------------+-------------------+-------------------+----------------+--------+
|count_movie_id|approx_count_movie_id|count_d_movie_id|       min_time_tag|    min_time_rating|max_count_rating|max_tags|
+--------------+---------------------+----------------+-------------------+-------------------+----------------+--------+
|         79477|                89110|           79477|2005-12-24 07:00:10|1995-01-09 05:46:44|          122296|    1350|
+--------------+---------------------+----------------+-------------------+-------------------+----------------+--------+



### describe

In [8]:
movies_df.describe().show()

+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|summary|         movie_id|               title|              year|        avg_rating|     stddev_rating|      count_rating|
+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|  count|            79477|               79477|             79235|             76718|             76718|             76718|
|   mean|153654.5569661663|                null|1995.6375717801477|3.0151092312105634|0.7114514194843852| 440.2630803722725|
| stddev|80063.84778983405|                null| 25.71829579663134|0.7784714800542165|0.4841102151081126|2921.2112388061987|
|    min|                1|"""BLOW THE NIGHT...|              1874|               0.5|               0.0|                 1|
|    max|            99999|       貞子3D (2012)|              2023|               5.0|              2.25|            122296|
+-

In [9]:
### se pueden enviar columnas - for all numerical or string columns
movies_df.describe(["year", "avg_rating", "stddev_rating", "count_rating", "min_time_tag", "min_time_rating"]).show()

+-------+------------------+------------------+------------------+------------------+
|summary|              year|        avg_rating|     stddev_rating|      count_rating|
+-------+------------------+------------------+------------------+------------------+
|  count|             79235|             76718|             76718|             76718|
|   mean|1995.6375717801477|3.0151092312105634|0.7114514194843852| 440.2630803722725|
| stddev| 25.71829579663134|0.7784714800542165|0.4841102151081126|2921.2112388061987|
|    min|              1874|               0.5|               0.0|                 1|
|    max|              2023|               5.0|              2.25|            122296|
+-------+------------------+------------------+------------------+------------------+



### summary

In [10]:
## Use summary for expanded statistics and control over which statistics to compute.
movies_df.summary().show()

+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|summary|         movie_id|               title|              year|        avg_rating|     stddev_rating|      count_rating|
+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|  count|            79477|               79477|             79235|             76718|             76718|             76718|
|   mean|153654.5569661663|                null|1995.6375717801477|3.0151092312105634|0.7114514194843852| 440.2630803722725|
| stddev|80063.84778983405|                null| 25.71829579663134|0.7784714800542165|0.4841102151081126|2921.2112388061987|
|    min|                1|"""BLOW THE NIGHT...|              1874|               0.5|               0.0|                 1|
|    25%|         103171.0|                null|              1981|              2.59|              0.25|                 2|


In [12]:
movies_df.summary(["count", "min", "25%", "75%", "max"]).show()

+-------+--------+--------------------+-----+----------+-------------+------------+
|summary|movie_id|               title| year|avg_rating|stddev_rating|count_rating|
+-------+--------+--------------------+-----+----------+-------------+------------+
|  count|   79477|               79477|79235|     76718|        76718|       76718|
|    min|       1|"""BLOW THE NIGHT...| 1874|       0.5|          0.0|           1|
|    25%|103171.0|                null| 1981|      2.59|         0.25|           2|
|    75%|211586.0|                null| 2015|       3.5|         1.06|          31|
|    max|   99999|       貞子3D (2012)| 2023|       5.0|         2.25|      122296|
+-------+--------+--------------------+-----+----------+-------------+------------+



### crossJoin

In [13]:
df = movies_df.summary()
df.show()

+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|summary|         movie_id|               title|              year|        avg_rating|     stddev_rating|      count_rating|
+-------+-----------------+--------------------+------------------+------------------+------------------+------------------+
|  count|            79477|               79477|             79235|             76718|             76718|             76718|
|   mean|153654.5569661663|                null|1995.6375717801477|3.0151092312105634|0.7114514194843852| 440.2630803722725|
| stddev|80063.84778983405|                null| 25.71829579663134|0.7784714800542165|0.4841102151081126|2921.2112388061987|
|    min|                1|"""BLOW THE NIGHT...|              1874|               0.5|               0.0|                 1|
|    25%|         103171.0|                null|              1981|              2.59|              0.25|                 2|


In [14]:
df_ids = spark.range(1,10,1)
df_ids.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [15]:
df.crossJoin(df_ids).count()

72

### sample

In [16]:
movies_df.count() * 0.00009

7.1529300000000005

In [17]:
movies_sample_df = movies_df \
    .select('movie_id', 'title', 'genres', 'year', 'min_time_tag', 'avg_rating', 'stddev_rating', 'count_rating', 'min_time_rating') \
    .sample(False, 0.00009, 1)
movies_sample_df.count()

9

In [18]:
movies_sample_df.show()

+--------+--------------------+--------------------+----+-------------------+----------+-------------+------------+-------------------+
|movie_id|               title|              genres|year|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+--------------------+--------------------+----+-------------------+----------+-------------+------------+-------------------+
|    6714|So Close (Chik Ye...|[Action, Comedy, ...|2002|2009-04-07 12:14:07|       3.3|         1.06|         171|2003-08-31 22:10:05|
|  127062|The Lumière Broth...|       [Documentary]|1996|2015-02-08 05:24:46|       3.3|         1.65|          10|2015-01-30 11:55:53|
|  139034|The Closed Circui...|      [Crime, Drama]|2013|2018-05-24 01:16:17|      3.41|         1.02|          11|2015-07-18 02:27:23|
|  141191|         OPA! (2005)|   [Comedy, Romance]|2005|               null|      2.83|         0.62|           3|2015-08-23 07:47:58|
|  179875| The Sex Trip (2017)|   [Comedy, Fanta

### na

In [19]:
first_movies_df = movies_df \
    .select('movie_id', 'title', 'genres', 'year', 'min_time_tag', 'avg_rating', 'stddev_rating', 'count_rating', 'min_time_rating') \
    .limit(5)
#first_movies_df.show(1)

In [20]:
id_df = spark.range(1,11,1).select(f.col("id").alias("movie_id"))

In [21]:
null_df = id_df.join(first_movies_df, ["movie_id"], "left")
null_df.show()

+--------+--------------------+--------------------+----+-------------------+----------+-------------+------------+-------------------+
|movie_id|               title|              genres|year|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+--------------------+--------------------+----+-------------------+----------+-------------+------------+-------------------+
|       1|    Toy Story (1995)|[Adventure, Anima...|1995|2006-01-12 19:19:35|      3.89|         0.93|       76813|1996-01-28 18:00:00|
|       2|      Jumanji (1995)|[Adventure, Child...|1995|2006-01-13 14:14:30|      3.28|         0.96|       30209|1996-01-28 18:00:00|
|       3|Grumpier Old Men ...|   [Comedy, Romance]|1995|2006-01-16 00:28:45|      3.17|         1.02|       15820|1996-01-29 17:54:45|
|       4|Waiting to Exhale...|[Comedy, Drama, R...|1995|2007-08-13 15:04:13|      2.87|         1.11|        3028|1996-02-01 08:33:45|
|       5|Father of the Bri...|            [Come

##### na.fill

In [22]:
# alias de fillna

# null_df.na.fill("Unknow").show()
# null_df.na.fill(0).show()
# null_df.na.fill(0, ["avg_rating", "stddev_rating", "count_rating"]).show()

# sólo funciona con los tipos: int, float, string, bool, no funciona con tipos complejos
#null_df.na.fill({"title": "Unknow", "year": 1970}).show()

##### na.replace

In [23]:
# alias de replace()

# null_df.na.replace(1995, 1900).show()
# null_df.na.replace([1995, 1, 2], [1970, 0, 0]).show()
# null_df.na.replace([1995, 1, 2, 3.89], [1970, 0, 0, 4], ["movie_id", "year"]).show()

# No se permite colocar múltiples tipos
# null_df.na.replace([1995, 1, 2, "Toy Story (1995)"], [1970, 0, 0, "Toy Story"], ["movie_id", "year"]).show() # -> error

#null_df.na.replace({1995:1970, 1:0, 2:0, 3.89:4}, subset=["movie_id", "year"]).show()

##### na.drop

In [24]:
# alias de dropna

drop_df = null_df \
    .na.fill(0) \
    .na.fill("Unknow")

# drop_df.show()

# drop_df.na.drop("any").show()
# drop_df.select("genres", "min_time_tag", "min_time_rating").na.drop("all").show()
# drop_df.na.drop("all", thresh=6).show()

# drop_df.na.drop("any", subset=["genres", "avg_rating", "stddev_rating"]).show()
# drop_df.na.drop("any", thresh=3, subset=["genres", "avg_rating", "stddev_rating"]).show()

### UDF (User Defined Functions)

In [34]:
from pyspark.sql.types import *

In [25]:
## Forma 1 - utilizando la funcion udf

def some_str_function(param_1: str, param_2: str) -> str:
    return param_1 + param_2

function_type1_udf = f.udf(lambda item_1, item_2: some_str_function(item_1, item_2), StringType())

movies_df \
    .select("movie_id", "title", function_type1_udf(f.col("movie_id"), f.col("title")).alias("concat")) \
    .show(1,False)

+--------+----------------+-----------------+
|movie_id|title           |concat           |
+--------+----------------+-----------------+
|1       |Toy Story (1995)|1Toy Story (1995)|
+--------+----------------+-----------------+
only showing top 1 row



In [26]:
## Forma 2 - utilizando la funcion udfcomo decorador
@f.udf(returnType=StringType())
def some_str_function_(param_1: str, param_2: str) -> str:
    return param_1 + param_2

movies_df \
    .select("movie_id", "title", some_str_function_(f.col("movie_id"), f.col("title")).alias("concat")) \
    .show(1,False)

+--------+----------------+-----------------+
|movie_id|title           |concat           |
+--------+----------------+-----------------+
|1       |Toy Story (1995)|1Toy Story (1995)|
+--------+----------------+-----------------+
only showing top 1 row



##### Utilizar UDF en sql

In [33]:
movies_df.createOrReplaceTempView("movies")

spark.table("movies").show(1)

+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|movie_id|           title|              genres|year|           tag_count|       min_time_tag|avg_rating|stddev_rating|count_rating|    min_time_rating|
+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
|       1|Toy Story (1995)|[Adventure, Anima...|1995|[{1990S, 1}, {200...|2006-01-12 19:19:35|      3.89|         0.93|       76813|1996-01-28 18:00:00|
+--------+----------------+--------------------+----+--------------------+-------------------+----------+-------------+------------+-------------------+
only showing top 1 row



In [30]:
spark.udf.register("CONCAT_UDF_V1", some_str_function)

spark.sql("select CONCAT_UDF_V1(movie_id, title) as concat, movie_id, title from movies").show(1)

+-----------------+--------+----------------+
|           concat|movie_id|           title|
+-----------------+--------+----------------+
|1Toy Story (1995)|       1|Toy Story (1995)|
+-----------------+--------+----------------+
only showing top 1 row



In [32]:
spark.udf.register("CONCAT_UDF_V2", function_type1_udf)

spark.sql("select CONCAT_UDF_V2(movie_id, title) as concat, movie_id, title from movies").show(1)

+-----------------+--------+----------------+
|           concat|movie_id|           title|
+-----------------+--------+----------------+
|1Toy Story (1995)|       1|Toy Story (1995)|
+-----------------+--------+----------------+
only showing top 1 row



##### UDF isn't null-safe!!

In [64]:
def sum_values(val_1, val_2):
    return val_1 + val_2

number_ops_udf = f.udf(lambda val_1, val_2: sum_values(val_1, val_2), DoubleType())

spark.udf.register("NUMBER_UPS_UDF", number_ops_udf)

spark \
    .sql("""select avg_rating, stddev_rating, NUMBER_UPS_UDF(avg_rating, stddev_rating) as sum_values from movies""") \
    .filter(f.col("avg_rating").isNull()) \
    #.show(10)

DataFrame[avg_rating: double, stddev_rating: double, sum_values: double]

In [63]:
def sum_values_null_safe(val_1, val_2):
    if (val_1 is None) | (val_2 is None):
        return 0.0
    return val_1 + val_2

number_ops_udf = f.udf(lambda val_1, val_2: sum_values_null_safe(val_1, val_2), DoubleType())

spark.udf.register("NUMBER_UPS_UDF", number_ops_udf)

spark \
    .sql("""select avg_rating, stddev_rating, NUMBER_UPS_UDF(avg_rating, stddev_rating) as sum_values from movies""") \
    .filter(f.col("avg_rating").isNull()) \
    .show(10)

+----------+-------------+----------+
|avg_rating|stddev_rating|sum_values|
+----------+-------------+----------+
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
|      null|         null|       0.0|
+----------+-------------+----------+
only showing top 10 rows

